In [16]:
import json
import numpy as np
import pandas as pd
from copy import copy
import sys
sys.path.append('/home/ndsviriden/MinMax94/src/utils') 
from constants import RUSSIAN_TIME_ZONES, rp5_columns, data_directory, ROAD_LAYERS, CATEGORY, \
        ROAD_CATEGORY, MAINTAINABILITY_LEVEL
from interpolation import interpolate_mmx, create_patterns
from converters import convert_rp5_to_mmx, convert_raw_to_mmx, convert_mmx_to_mmcc_forecast, convert_raw_to_mmx, \
                convert_mmx_to_mmcc_rwis
from loaders import load_rp5_stations, load_mm94_stations
from geographical import find_nearest_wmo_station


## Loading station data

In [177]:
mm94_station_id = 1821

raw = load_mm94_stations(mm94_station_id)
mmx_rwis = convert_raw_to_mmx(raw)
mmx_rwis_interpolated = interpolate_mmx(mmx_rwis)
mmx = create_patterns(mmx_rwis_interpolated)
roadcast = copy(mmx) #.set_index("date_time_utc")
roadcast.head()

,date_time_utc,data_freezing_point,id_dampness,id_t_air,data_precip_code,data_cloudiness,data_wind_dir,id_wind_velocity,data_salinity,id_t_underroad,...,data_t_road,data_dampness,id_dew_point,id_precip_code,data_precip_count,id_cloudiness,id_pressure,id_salinity,data_t_underroad,id_freezing_point
12,2008-03-23 21:00:00,NaN,6998431.0,6998435.0,NaN,NaN,NaN,NaN,NaN,6998436.0,...,-3.377778,98.0,6998434.0,NaN,NaN,6998430.0,NaN,NaN,-3.666667,NaN
13,2008-03-23 21:30:00,NaN,6998431.0,6998435.0,NaN,NaN,NaN,NaN,NaN,6998436.0,...,-3.355556,98.1,6998434.0,NaN,NaN,6998430.0,NaN,NaN,-3.633333,NaN
14,2008-03-23 22:00:00,NaN,6998431.0,6998435.0,NaN,NaN,NaN,NaN,NaN,6998436.0,...,-3.333333,98.2,6998434.0,NaN,NaN,6998430.0,NaN,NaN,-3.600000,NaN
15,2008-03-23 22:30:00,NaN,6998431.0,6998435.0,NaN,NaN,NaN,NaN,NaN,6998436.0,...,-3.311111,98.3,6998434.0,NaN,NaN,6998430.0,NaN,NaN,-3.566667,NaN
16,2008-03-23 23:00:00,NaN,6998938.0,6998942.0,NaN,NaN,NaN,NaN,NaN,6998943.0,...,-3.288889,98.4,6998941.0,NaN,NaN,6998937.0,NaN,NaN,-3.533333,NaN


In [178]:
rain_mask = roadcast['data_precip_code'] == 10
snow_mask = roadcast['data_precip_code'] == 20
rain_snow_mask = roadcast['data_precip_code'] == 30

roadcast['data_rain_intensity'] = rain_mask * roadcast['data_precip_count'] + \
                            rain_snow_mask * roadcast['data_precip_count'] / 2
    
roadcast['data_snow_intensity'] = snow_mask * roadcast['data_precip_count'] + \
                        rain_snow_mask * roadcast['data_precip_count'] / 2

## Loose snow

In [179]:
from constants import MmxColumns

is_t_road_below_zero = roadcast[MmxColumns.ROAD_TEMPERATURE] < 0

is_snow = roadcast['data_snow_intensity'] > 0

is_rain = roadcast['data_rain_intensity'] > 0

is_t_air_for_loose_snow = (((roadcast[MmxColumns.AIR_TEMPERATURE] <= -6) \
                                & (-10 <= roadcast[MmxColumns.AIR_TEMPERATURE]) \
                                & (roadcast[MmxColumns.HUMIDITY] <= 90)) \
                            | (roadcast[MmxColumns.AIR_TEMPERATURE] < -10))

is_loose_snow = is_t_road_below_zero & is_snowfall & is_t_air_for_loose_snow

roadcast['state_heavy_loose_snow'] = is_loose_snow & (roadcast['data_snow_intensity'] >= 3)
roadcast['state_light_loose_snow'] = is_loose_snow & (roadcast['data_snow_intensity'] < 3)


In [180]:
is_t_air_for_packed_snow = (roadcast[MmxColumns.AIR_TEMPERATURE] > -6) \
                            | ((roadcast[MmxColumns.AIR_TEMPERATURE] >= -10)
                            & (roadcast[MmxColumns.HUMIDITY] > 90))
    
roadcast['state_packed_snow'] = is_t_road_below_zero & is_snowfall & is_t_air_for_packed_snow

In [96]:
# TODO: is_frost
# roadcast['state_frost'] = ...

In [181]:
roadcast['state_sleet'] = is_t_road_below_zero & is_rain

In [182]:
is_not_sleet = ~roadcast['state_sleet']

snow_history_avg = roadcast.set_index(MmxColumns.DATE_TIME_UTC)['data_snow_intensity'].rolling('4h').mean()
rain_history_avg = roadcast.set_index(MmxColumns.DATE_TIME_UTC)['data_rain_intensity'].rolling('4h').mean()
road_history_avg = roadcast.set_index(MmxColumns.DATE_TIME_UTC)[MmxColumns.ROAD_TEMPERATURE].rolling('4h').mean()

is_snow_history = snow_history_avg > 0
is_rain_history = rain_history_avg > 0
is_t_road_history_above_zero = road_history_avg > 0

roadcast['state_ice_crusted_ground'] = is_not_sleet & is_t_road_below_zero  & \
                                    is_t_road_history_above_zero & is_snow_history # | is_snow_history) 

/usr/local/lib/python3.5/dist-packages/pandas/core/indexes/base.py:3033: RuntimeWarning: Cannot compare type 'Timestamp' with type 'int', sort order is undefined for incomparable objects
  return this.join(other, how=how, return_indexers=return_indexers)


In [ ]:
def _is_frost(self):
        """
        Check whether it is suitable conditions for frost on the road
        :return: boolean
        """
        if self._is_frost_ending_point(self.roadcast):
            historical_records = self.historical_roadcast_maker.get_previous_three_timeline_records(
                self.roadcast[Mmcc.TIMESTAMP])
            for i in range(len(historical_records)):
                if self._is_frost_beginning_point(historical_records[i]) and self._is_frost_middle_point(
                        historical_records[i:] + [self.roadcast]):
                    return True
        return False

    @staticmethod
    def _is_frost_ending_point(point):
        if point[Mmcc.SNOW_INTENSITY] == 0 and point[Mmcc.RAIN_INTENSITY] == 0:
            if point[Mmcc.ROAD_TEMPERATURE] < 0:
                return True
        return False

    @staticmethod
    def _is_frost_beginning_point(point):
        if point[Mmcc.SNOW_INTENSITY] == 0 and point[Mmcc.RAIN_INTENSITY] == 0:
            if point[Mmcc.ROAD_TEMPERATURE] >= point[Mmcc.DEW_POINT_TEMPERATURE]:
                return True
        return False

    @staticmethod
    def _is_frost_middle_point(points):
        x0_dt = datetime.strptime(points[0][Mmcc.TIMESTAMP], API_TIME_FORMAT)

        road_temp_points = []
        dew_point_temp_points = []

        for point in points:
            x_dt = datetime.strptime(point[Mmcc.TIMESTAMP], API_TIME_FORMAT)
            x_x0 = (x_dt - x0_dt).total_seconds() / 60
            x = x_x0
            road_temp_points.append(Point(x=x, y=point[Mmcc.ROAD_TEMPERATURE]))
            dew_point_temp_points.append(Point(x=x, y=point[Mmcc.DEW_POINT_TEMPERATURE]))

        road_line = PolyLine(road_temp_points)
        dew_point_line = PolyLine(dew_point_temp_points)

        road_dew_crossing_points = get_polylines_crossing_points(road_line, dew_point_line)

        if not road_dew_crossing_points or road_dew_crossing_points[0].y >= 0:
            return False
return True